In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import sys

In [2]:
dir_current = os.path.abspath('')
dir_parent  = os.path.dirname(dir_current)
if not dir_parent in sys.path: sys.path.append(dir_parent)

In [3]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Loading Custom Dataset

In [ ]:
# Downloading The Dataset
ds = tfds.load('deep_weeds', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

In [4]:
x_train, y_train = tfds.as_numpy(tfds.load('deep_weeds', split='train[:80%]', batch_size=-1, as_supervised=True))
x_valid, y_valid = tfds.as_numpy(tfds.load('deep_weeds', split='train[80:85%]', batch_size=-1, as_supervised=True))
x_test,  y_test  = tfds.as_numpy(tfds.load('deep_weeds', split='train[85%:]', batch_size=-1, as_supervised=True))

## Creating DataSet Objects

In [5]:
Nclasses = 9
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth = Nclasses)
def normalize(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

In [6]:
from functions import VGG_resize

bs = 32

# Training
#-----------------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])
train_dataset = train_dataset.map(to_categorical)
train_dataset = train_dataset.map(normalize)
train_dataset = train_dataset.map(VGG_resize)
# Divide in batches
train_dataset = train_dataset.batch(bs)
# Repeat
train_dataset = train_dataset.repeat()

#Validation   
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# Enconding
valid_dataset = valid_dataset.map(to_categorical)
# Normalize
valid_dataset = valid_dataset.map(normalize)
# Resize
valid_dataset = valid_dataset.map(VGG_resize)
# Divide in batches
valid_dataset = valid_dataset.batch(bs)
# Repeat
valid_dataset = valid_dataset.repeat()

#Testing 
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.map(normalize)
test_dataset = test_dataset.map(VGG_resize)
test_dataset = test_dataset.batch(1)

## Creating VGG Model

In [7]:
from models import VGG16_body

input_layer   = tf.keras.Input((224,224,3))
output_layer  = VGG16_body(input_layer, Quantization = False,N_labels = Nclasses)

VGG16  = tf.keras.Model(inputs=input_layer, outputs=output_layer)
VGG16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
lambda (Lambda)              (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
lambda_1 (Lambda)            (None, 224, 224, 64)      0         
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 224, 224, 64)]    0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928 

## Training Options

In [8]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
VGG16.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Callbacks

In [9]:
early_stop = True
cwd = os.getcwd()
callbacks = []

# Early Stopping
# --------------
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    callbacks.append(es_callback)


## Training

In [10]:
VGG16.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(x_valid.shape[0] / bs)), 
          callbacks=callbacks)

Train for 438 steps, validate for 28 steps
Epoch 1/100
438/438 [==============================] - 215s 490ms/step - loss: 1.6021 - accuracy: 0.5219 - val_loss: 1.5399 - val_accuracy: 0.4977
Epoch 2/100
438/438 [==============================] - 198s 451ms/step - loss: 1.4389 - accuracy: 0.5309 - val_loss: 1.4429 - val_accuracy: 0.5080
Epoch 3/100
438/438 [==============================] - 196s 448ms/step - loss: 1.2761 - accuracy: 0.5562 - val_loss: 1.2681 - val_accuracy: 0.5525
Epoch 4/100
438/438 [==============================] - 197s 450ms/step - loss: 1.1455 - accuracy: 0.5863 - val_loss: 1.1864 - val_accuracy: 0.5811
Epoch 5/100
438/438 [==============================] - 196s 448ms/step - loss: 1.0616 - accuracy: 0.6134 - val_loss: 1.0785 - val_accuracy: 0.5959
Epoch 6/100
438/438 [==============================] - 196s 449ms/step - loss: 0.9934 - accuracy: 0.6399 - val_loss: 1.0769 - val_accuracy: 0.6050
Epoch 7/100
438/438 [==============================] - 196s 448ms/step - lo

## Saving Weights

In [14]:
Wgt_dir = os.path.join(cwd, 'TrainedWeights')
Wgt_dir = os.path.join(cwd, 'WeedWeights')
Wgt_dir = os.path.join(Wgt_dir, 'Weights')
VGG16.save_weights(Wgt_dir)